In [1]:
import geogridfusion
import pvdeg
import pvlib

In [2]:
conn = geogridfusion.start()

Starting Postgres subprocess...
PostgreSQL connection established after 2.70 seconds.
postgis already installed


In [ ]:
import geogridfusion.queries


geogridfusion.queries.rows_by_id(conn=conn, ids=[1, 2, 3, 4])

In [ ]:
geogridfusion.initialize_tables(conn)

In [ ]:
sw, sm = pvlib.iotools.get_solrad(station="abq", start="2022-01-01", end="2022-01-05")

In [ ]:
geogridfusion.store_single(
    conn=conn,
    weather_df=pvdeg.weather.map_weather(sw),
    meta=pvdeg.weather.map_meta(sm),
    source_name="solrad",
    tmy=False,
)

In [ ]:
client = pvdeg.geospatial.start_dask()

coords = [(45 + i, -115 + k) for i in range(5) for k in range(5)]

geo_weather, geo_meta, failed = pvdeg.weather.weather_distributed(
    database="PVGIS", coords=coords
)

client.close()

In [ ]:
weather, meta = pvdeg.weather.get(database="PVGIS", id=(45, -115))

In [ ]:
geogridfusion.store_single(
    conn=conn, weather_df=weather, meta=meta, tmy=True, source_res="pvgis"
)

When we run the cell below, we see that we will get a collsion from the data inserted above.

We want to rewrite this function so it can be done async or with multiprocessing.

In [ ]:
for i in range(25):
    w = geo_weather.isel(gid=i).drop_vars(("gid",)).to_pandas()
    m = geo_meta.iloc[i].to_dict()

    geogridfusion.store_single(
        conn=conn, weather_df=w, meta=m, tmy=True, source_name="pvgis"
    )

We can easily get one output from what we have written so far. We need to be able to read MANY into dataset form.

In [ ]:
geogridfusion.sources(conn)

In [ ]:
lw, lm = geogridfusion.load_single(
    conn=conn,
    latitude=sm["latitude"] + 10,
    longitude=sm["longitude"],
    source_name="solrad",
)

lw.shape

In [7]:
gw, gm = geogridfusion.load_many(
    conn=conn,
    source_name="pvgis",
    spatial_search=True,
    spatial_search_distance_floor=140 * 1000,
    spatial_search_latitude0=45,
    spatial_search_longitude0=-114,
)

In [8]:
gm

,latitude,longitude,irradiance_time_offset,altitude,wind_height,Source
3,45.0,-114.0,0.0,2076.0,10,PVGIS
5,45.0,-112.0,0.0,2219.0,10,PVGIS
12,47.0,-115.0,0.0,1515.0,10,PVGIS
14,47.0,-113.0,0.0,1281.0,10,PVGIS
23,49.0,-114.0,0.0,1549.0,10,PVGIS
26,49.0,-111.0,0.0,1056.0,10,PVGIS


In [ ]:
gm